In [281]:
import pandas as pd
import sys
sys.path.append('../src')
# import importlib
# import statistics
# importlib.reload(statistics)
from sales_statistics import get_col_stats_filename

In [282]:
# !NOTE - make sure you uploaded data file into appropriate folder 
filename = '../data/raw/rossman_prepared.csv'
raw_df = pd.read_csv(filename)
target_col = 'Sales'

In [283]:
open_df = raw_df[raw_df['Open'] == 1]

In [284]:
statistic_setups = [
    {
        'group_by': 'Month',
        'values_map': {
            1: "January", 2: "February", 3: "March", 4: "April",
            5: "May", 6: "June", 7: "July", 8: "August",
            9: "September", 10: "October", 11: "November", 12: "December"
        }
    },
    {
        'group_by': 'DayOfWeek',
        'values_map': {
            1: "Monday", 2: "Tuesday", 3: "Wednesday", 4: "Thursday",
            5: "Friday", 6: "Saturday", 7: "Sunday"
        }
    }
]

In [285]:
for setup in statistic_setups:
    group_by_col = setup['group_by']
    filename = get_col_stats_filename(group_by_col)
    values_map = setup['values_map']
    df_avg = open_df.groupby(group_by_col, as_index=False)['Sales'].mean().round(2)
    df_avg.rename(columns={'Sales': 'Average_Sales'}, inplace=True)
    df_avg[group_by_col] = df_avg[group_by_col].map(values_map)

    # Save df_avg to a CSV file
    df_avg.to_csv('../data/streamlit/' + filename, index=False)

In [280]:
user_inpute_cols = [
    'Date', 'Promo', 'CompetitionDistance',
    'StateHoliday', 'SchoolHoliday', 'StoreType',
    'Assortment', 'Promo2', 'Promo2SinceWeek',
    'Promo2SinceYear', 'PromoInterval', 'CompetitionOpenSinceMonth',
    'CompetitionOpenSinceYear', 'DaysAfterHoliday', 'DaysBeforeHoliday',
    'Sales_Lag1', 'Sales_Lag2', 'Sales_Lag3', 'Sales_Lag7', 'Sales_Lag14',
    'Sales_Lag30', 'Customers_Lag1', 'Customers_Lag7', 'SalesPerCustomer_Lag1']

In [259]:
# Prepare summary
summary = []

open_df.loc[:, 'Date'] = pd.to_datetime(open_df['Date'])


for col in user_inpute_cols:
    col_data = open_df[col]
    col_type = col_data.dtype
    
    # Detect boolean-like numeric
    if np.issubdtype(col_type, np.number) and set(col_data.unique()).issubset({0, 1}):
        summary.append({
            "Column": col,
            "Type": "boolean",
            "Min": None,
            "Max": None,
            "Unique_Values": [0, 1]
        })
    
    # Detect datetime
    elif np.issubdtype(col_type, np.datetime64):
        summary.append({
            "Column": col,
            "Type": "date",
            "Min": col_data.min(),
            "Max": col_data.max(),
            "Unique_Values": None
        })
    
    # Numeric
    elif np.issubdtype(col_type, np.number):
        summary.append({
            "Column": col,
            "Type": "numeric",
            "Min": col_data.min(),
            "Max": col_data.max(),
            "Unique_Values": None
        })
    
    # Categorical / object
    else:
        summary.append({
            "Column": col,
            "Type": "categorical",
            "Min": None,
            "Max": None,
            "Unique_Values": col_data.unique().tolist()
        })

# Convert to DataFrame
summary_df = pd.DataFrame(summary)


# Save summary_df to a CSV file
summary_df.to_csv('../data/streamlit/user_inputs_data.csv', index=False)